#This is my Google Colab script for training and evaluation of the models.; Faster R-CNN, SSD MobileNet Resnet 50 + 152.


In [ ]:
#Mounting data from Tensorflow folder saved in my Google Drive.

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Updating libraries required for training and evaluation of models.


!apt-get update --fix-missing
!pip install -q pycocotools
!pip install -q Cython contextlib2 pillow lxml matplotlib
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

In [ ]:
#Setting up the Protobuff libraries that will also be utilised.

%cd '/content/gdrive/My Drive/TensorFlow/models/research/'
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
#creating the environment that will be used for training/evaluation.

import os
import sys
os.environ['PYTHONPATH']+=":/content/gdrive/My Drive/TensorFlow/models"
sys.path.append("/content/gdrive/My Drive/TensorFlow/models/research")

In [ ]:
#running the built in tensorflow file  - setup.py

!python setup.py build
!python setup.py install

In [ ]:
#This command shows us the gpu that is being used in the current Colab Environment.
#Shows the specific gpu mopdel and the amnount of VRAM available for use. Usually TESLA T4

!nvidia-smi

In [ ]:
#Testing the installation of tensorflow on the Colab system and also importing the object detection modules.

%cd '/content/gdrive/My Drive/TensorFlow/models/research/object_detection/builders/'
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

In [ ]:
#training cell for each model.

#Models used in training:
# my_faster_rcnn_resnet152_v1_800x1333_coco17_tpu-8
# my_faster_rcnn_resnet50_v1_800x1333_coco17_tpu-8
# my_ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8
# my_ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8

%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'
!python model_main_tf2.py --model_dir=models/my_faster_rcnn_resnet152_v1_800x1333_coco17_tpu-8 --pipeline_config_path=models/my_faster_rcnn_resnet152_v1_800x1333_coco17_tpu-8/pipeline.config

In [ ]:
# Opening up tensorboard to monitor the progress of the models being trained and also monitoring the evaluation of each model.

#move into working directory.
%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'
#load tensorboard and specify folder for evaluation
%load_ext tensorboard
%tensorboard --logdir=models/my_faster_rcnn_resnet152_v1_800x1333_coco17_tpu-8

In [ ]:
#This is where evaluation of the checkpoints from each trained model occurs. 
#Can change which checkpoint is evaluated by editing "modal_checkpoint path" value in the checkpoint file in each model.

%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'
!python model_main_tf2.py --model_dir=exported-models/faster_rcnn_resnet152_final --pipeline_config_path=exported-models/faster_rcnn_resnet152_final/pipeline.config --checkpoint_dir=exported-models/faster_rcnn_resnet152_final/checkpoint

In [ ]:
#Exporting the completed trained model and saving to exported-model

%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_faster_rcnn_resnet152_v1_800x1333_coco17_tpu-8/pipeline.config --trained_checkpoint_dir ./models/my_faster_rcnn_resnet152_v1_800x1333_coco17_tpu-8/ --output_directory ./exported-models/faster_rcnn_resnet152_final

In [ ]:
#using the completed models for object detection on test images

%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'

#importing libraries
import numpy as np
import tensorflow as tf
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# all the completed models for inference

#'/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/faster_rcnn_final/saved_model'
#'/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/ssd_resnet50_final/saved_model'
#'/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/ssd_resnet152_final/saved_model'


#importing the assets required for inference (models used for inference)
final_model_path = "exported-models/ssd_resnet152_final/saved_model"
loaded_model = tf.saved_model.load(final_model_path)

print("Model loading completed")




In [ ]:
#loading label map and converting into correct format for use in inference

label_map_path = '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt'

# category index is an dict of catergory definitions for each class (id and name). in this case the class of each vehicle present in the images.
category_index = label_map_util.create_category_index_from_labelmap(label_map_path, use_display_name=True)


In [ ]:
#Loading the test images that will be used for inference.

##Examples for images folder

# SSDB00479.JPG - one car
# SSDB00569.JPG - lots of cars
# SSDB01287.JPG - lots of cars on a street
# SSDB01659.JPG - two vans
# 201936973_a7f65606a8_b.jpg - bus and cars
# SSDB01634.JPG - truck and cars
# SSDB01526.JPG - one van
# SSDB01677.JPG - one van and cars
# SSDB00442.JPG - multiple cars driving

##########################


##Final test images 

# "images/final/1.jpg","images/final/2.jpg","images/final/3.jpg","images/final/4.jpg","images/final/5.jpg","images/final/6.jpg","images/final/7.jpg","images/final/8.jpg","images/final/9.jpg","images/final/10.jpg","images/final/11.jpg","images/final/12.jpg"
# "images/final/13.JPG","images/final/14.JPG","images/final/15.JPG","images/final/16.JPG","images/final/17.JPG","images/final/18.JPG","images/final/19.JPG","images/final/20.JPG","images/final/21.JPG","images/final/22.JPG","images/final/23.JPG","images/final/24.JPG","images/final/25.JPG"


#test_images = ["images/test/SSDB01677.JPG","images/test/SSDB01659.JPG", "images/test/SSDB00442.JPG", "images/test/SSDB01287.JPG"]
test_images = ["images/final/13.JPG","images/final/14.JPG","images/final/15.JPG","images/final/16.JPG","images/final/17.JPG","images/final/18.JPG","images/final/19.JPG","images/final/20.JPG","images/final/21.JPG","images/final/22.JPG","images/final/23.JPG","images/final/24.JPG","images/final/25.JPG"]
print(test_images)

In [ ]:
#@title
#Running the actual detection
# This code allows me to utilise the final models that have been trained on a new dataset of images and compare performance using the results from this testing.


%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'

#loads the images in the test_images list above into an numpy array
def image_numpy_array(path):
    return np.array(Image.open(path))

for image in test_images:
  image_numpy=image_numpy_array(image)

  #Here the images used converted into tensors which are required for the detection models.
  tensor_convert=tf.convert_to_tensor(image_numpy)
  #allows for use of multiple images at once.
  tensor_convert=tensor_convert[tf.newaxis,...]

  model_detection=loaded_model(tensor_convert)


  num_detections=int(model_detection.pop('num_detections'))
  model_detection={key:value[0,:num_detections].numpy()
                    for key,value in model_detection.items()}
  model_detection['num_detections']=num_detections

  model_detection['detection_classes']= model_detection['detection_classes'].astype(np.int64)
  
  image_np_with_detections=image_numpy.copy()

  #configuration of the image to be shown
  viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,         # the image to be shown
            model_detection['detection_boxes'],    # shows the box for each prediction.
            model_detection['detection_classes'],  # shows the class of each prediction.
            model_detection['detection_scores'], #shows the accuracy score on each box.
            category_index,           #using the label map categories
            use_normalized_coordinates=True, # whether the boxes are also interpreted as coordinates.
            line_thickness = 7,      # changes thickness of the lines.
            max_boxes_to_draw=100,   #max number of boxes that will be drawn  .
            min_score_thresh=.6,  #will only print boxes where accuracy is above 60%.
            agnostic_mode=False)

  print('Completed Detection for {}... '.format(image), end='')


  # printing out the figures after configuration above.
  %matplotlib inline
  plt.figure()

  # larger image printed out
  plt.figure(figsize=(40,20)) 
  plt.imshow(image_np_with_detections)
  plt.show()
  